In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [66]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
HAM_PROB_FILE = 'SpamData/03_Testing/prob-ham.txt'
ALL_PROB_FILE = 'SpamData/03_Testing/prob-all.txt'

TEST_FEATURE_FILE = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_test_data[:5]

array([[ 8,  7,  1,  1],
       [ 8,  8,  1,  1],
       [ 8, 12,  1,  1],
       [ 8, 21,  1,  2],
       [ 8, 26,  1,  1]])

In [28]:
sparse_train_data[10][0]

0

## Create full matrix from sparse matrix

In [77]:
def generate_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
        
    full_matrix.set_index('DOC_ID', inplace = True) 
    return full_matrix

In [31]:
full_train_data = generate_full_matrix(sparse_train_data, VOCAB_SIZE)

In [32]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


## Calculating the Probability of Spam

In [33]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
prob_spam

0.3111609367214748

## Total Number of Words / Tokens

In [34]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,2,2,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5791,2,0,1,1,0,1,2,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [37]:
email_lengths = full_train_features.sum(axis=1)
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [36]:
total_wc = email_lengths.sum()
total_wc

444982

## Number of Tokens in Spam and Ham Emails

In [38]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1249,)

In [39]:
spam_wc = spam_lengths.sum()
spam_wc

193998

In [40]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [41]:
ham_wc = ham_lengths.sum()
ham_wc

250984

In [42]:
spam_wc + ham_wc - total_wc

0

## Summing the Tokens Occuring in Spam

In [43]:
full_train_features.shape

(4014, 2500)

In [45]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,2,0,0,3,2,2,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,2,0,1,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
train_spam_tokens.shape

(1249, 2500)

In [52]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1     ### Laplace smoothing
summed_spam_tokens.shape

(2500,)

In [53]:
summed_spam_tokens.tail()

2495    18
2496     7
2497    32
2498     4
2499     5
dtype: int64

## Summing the Tokens Occuring in Ham

In [54]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,2,2,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5791,2,0,1,1,0,1,2,0,1,2,...,0,0,0,0,0,0,0,0,0,0
5794,2,2,1,0,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5795,1,1,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [55]:
train_ham_tokens.shape

(2765, 2500)

In [56]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1     ### Laplace smoothing
summed_ham_tokens.shape

(2500,)

In [57]:
summed_ham_tokens.tail()

2495     9
2496    22
2497     3
2498    27
2499    36
dtype: int64

## P(Token | Spam) - Probability that a token occurs given the Email is Spam

In [58]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)    ## Adding VOCAB_SIZE to balance Laplace smoothing
prob_tokens_spam[:5]

0    0.009064
1    0.004926
2    0.006880
3    0.010662
4    0.006916
dtype: float64

In [59]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a token occurs given the Email is Non-Spam

In [60]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE)    ## Adding VOCAB_SIZE to balance Laplace smoothing
prob_tokens_ham[:5]

0    0.020806
1    0.009867
2    0.008044
3    0.003728
4    0.006300
dtype: float64

In [61]:
prob_tokens_ham.sum()

1.0

## P(Token) - Probability that a token occurs

In [62]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0

## Save the trained model

In [65]:
np.savetxt(SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(ALL_PROB_FILE, prob_tokens_all)

## Prepare test data

In [71]:
sparse_test_data.shape

(110577, 4)

In [78]:
full_test_data = generate_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

In [79]:
full_test_data.shape

(1723, 2501)

In [80]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [81]:
X_test.shape

(1723, 2500)

In [82]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_FILE, X_test)